# Model Comparison for Robot Policy Performance

This notebook compares the performance of different models tested using the `test_policy_client_mse.py` script.

In [1110]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import pandas as pd
import seaborn as sns
from typing import Dict, List, Any

# Set plot style
plt.style.use('ggplot')
sns.set_context("talk")

In [ ]:
# Define paths to model action files
# model_paths = {
#     "openvla": "results/actions_openvla_20250421_152240.pkl",
#     "o4-mini": "results/actions_o4-mini_20250421_152435.pkl",
#     "gpt-4o": "results/actions_gpt-4o_20250421_180623.pkl",
#     # "gpt-4o-tunnel": "results/actions_gpt-4o-tunnel_20250423_134722.pkl",
#     # "gpt-4o-tunnel-async": "results/actions_gpt-4o-tunnel_20250423_140019.pkl",
#     # "gpt-4o-mini": "results/actions_gpt-4o-mini_20250423_152620.pkl",
#     # "gpt-4o-mini-tunnel-async": "results/actions_gpt-4o-mini-async_20250423_140435.pkl",
#     # "gpt-4o-mini-async": "results/actions_gpt-4o-mini_20250423_141040.pkl",
#     # "gemini-2.0-flash-tunnel": "results/actions_gemini-2.0-flash-tunnel_20250423_142234.pkl",
#     "o3": "results/actions_o3_20250421_181454.pkl",
#     "gemini-2.5-pro": "results/actions_gemini-2-5-pro_20250421_172434.pkl",
#     "gemini-2.5-pro-history1": "results/actions_gemini-2-5-pro-history1_20250429_151242.pkl",
#     # "gemini-2.5-flash": "results/actions_gemini-2.5-flash_20250421_180103.pkl",
#     # "gemini-2.5-flash-history1": "results/actions_gemini-2-5-flash-history1_20250429_154648.pkl",
#     "gemini-2.5-pro-historytest": "results/actions_testhist_20250501_221849.pkl",
#     "gemini-2.5-pro-historytest2": "results/actions_testhist_20250501_222835.pkl",
#     "gemini-2-5-pro-historytest3-lang": "results/actions_testhist_lang_20250501_230050.pkl",
#     # "gemini-2.5-flash-tunnel": "results/actions_gemini-2.5-flash-tunnel_20250423_133919.pkl",
#     "gemini-2-0-flash": "results/actions_gemini-2-0-flash_20250422_140401.pkl",
#     # "gemini-2.0-flash-tunnel-async": "results/actions_gemini-2.0-flash-tunnel-async_20250423_142053.pkl",
#     "claude-3-5-sonnet": "results/actions_claude-3-5-sonnet_20250422_135539.pkl",
#     "claude-3-7-sonnet": "results/actions_claude-3-7-sonnet_20250422_141227.pkl",
#     # "openvla-tunnel": "results/actions_openvla-tunnel_20250423_110538.pkl",
#     # "openvla-tunnel2": "results/actions_openvla-tunnel2_20250423_110927.pkl"
#     # Add more models as needed
#     "ecot": "results/actions_ecot-tunnel_20250424_173038.pkl"
# }

def calc_expected_frames(history_length: int, history_choice: str) -> int:
    if history_choice == "all":
        return history_length
    elif history_choice == "last":
        return 1
    elif history_choice == "first":
        return 1
    elif history_choice == "alternate":
        return history_length // 2
    elif history_choice == "third":
        return history_length // 3
    else:
        raise ValueError(f"Invalid history_choice: {history_choice}")

RESULTS_DIR = "results"
RESULTS = dict()
for model_name in os.listdir(RESULTS_DIR):
    if os.path.isdir(os.path.join(RESULTS_DIR, model_name)):
        print(f"checking model {model_name}")
        if model_name.lstrip("history_sweep_") not in RESULTS:
            RESULTS[model_name.lstrip("history_sweep_")] = list()
        for filename in os.listdir(os.path.join(RESULTS_DIR, model_name)):
            if 'metrics' in filename and filename.endswith('.pkl'):
                print(f"found metrics file {filename}")
                metrics_dict = pickle.load(open(os.path.join(RESULTS_DIR, model_name, filename), "rb"))
                actions_file = filename.replace('metrics', 'actions')
                actions_dict = pickle.load(open(os.path.join(RESULTS_DIR, model_name, actions_file), "rb"))
                info_dict = {k: metrics_dict[k] for k in ['timestamp', 'model_name', 'history_length', 'history_choice']}
                info_dict['model_name'] = model_name.lstrip("history_sweep_")
                RESULTS[model_name.lstrip("history_sweep_")].append({"info": info_dict, "actions": actions_dict, "metrics": metrics_dict})


In [ ]:
def info_dict_to_title(info_dict):
    return f"{info_dict['model_name']}" + (f"-histlen-{info_dict['history_length']}-histchoice-{info_dict['history_choice']}" if info_dict['history_length'] is not None else f"")

# Check which models were loaded successfully
print(f"Successfully loaded data for {len(RESULTS)} models and {sum([len(v) for v in RESULTS.values()])} results")
for model_name, results_list in RESULTS.items():
    print(model_name)
    for result in results_list:
        print(f"- {info_dict_to_title(result['info'])}")

In [1113]:
# Function to compute metrics for models without pre-computed metrics
def compute_metrics(pred_actions, gt_actions):
    results = {}
    
    # MSE - Mean Squared Error
    results["mse"] = np.mean(np.square(pred_actions - gt_actions))
    
    # MAE - Mean Absolute Error
    results["mae"] = np.mean(np.abs(pred_actions - gt_actions))
    
    # Normalized MSE - divide by variance of ground truth
    gt_var = np.var(gt_actions)
    if gt_var > 0:
        results["nmse"] = results["mse"] / gt_var
    else:
        results["nmse"] = float('inf')
    
    # Action magnitude comparison
    results["pred_mag"] = np.mean(np.linalg.norm(pred_actions, axis=1))
    results["gt_mag"] = np.mean(np.linalg.norm(gt_actions, axis=1))
    results["mag_ratio"] = results["pred_mag"] / results["gt_mag"] if results["gt_mag"] > 0 else float('inf')
    
    return results

In [1114]:
# Extract and organize metrics across models for comparison
def extract_comparison_metrics(model_data: dict[str, list[dict]], force_compute: bool = True, min_results: int = 4):
    comparison = {}
    
    for model_name, model_results in model_data.items():
        for data in model_results:
            model_title = info_dict_to_title(data['info'])
            if model_title in comparison:
                copies = [x for x in comparison.keys() if model_title == x or (model_title in x and len(x)- 2 == len(model_title))]
                model_title = f"{model_title}-{len(copies)}"
            print(f"adding {model_title} to comparison")
            # If we have pre-computed metrics, use those
            if 'metrics' in data and 'avg_metrics' in data['metrics'] and not force_compute:
                comparison[model_title] = data['metrics']['avg_metrics']
            else:
                # Otherwise, compute metrics from actions
                print(f"Computing metrics for {model_title}...")
                all_results = []
                
                for traj in data['actions']:
                    pred_actions = traj['pred_actions']
                    gt_actions = traj['gt_actions']
                    
                    results = compute_metrics(pred_actions, gt_actions)
                    
                    # Add timing information if available
                    if 'inference_times' in traj:
                        results['mean_inference_time'] = np.mean(traj['inference_times'])
                    all_results.append(results)
                
                # Compute average metrics across all trajectories
                if len(all_results) < min_results:
                    print(f"Not enough results for {model_title}, skipping {len(all_results)} results")
                    continue
                metrics = {k: np.mean([r[k] for r in all_results]) for k in all_results[0]}
                # add info to the df
                metrics['model_name'] = model_name
                metrics['model_title'] = model_title
                metrics['history_length'] = data['info']['history_length'] if data['info']['history_length'] is not None else 0
                metrics['history_choice'] = data['info']['history_choice'] if data['info']['history_choice'] is not None else 'None'
                metrics['expected_frames'] = calc_expected_frames(data['info']['history_length'], data['info']['history_choice']) if data['info']['history_length'] is not None else 0
                comparison[model_title] = metrics

    return comparison

def consolidate_reruns(comparison: dict[str, dict[str, Any]]) -> dict[str, dict[str, Any]]:
    averaged_comparison = dict()
    multiples = [t for t in comparison.keys() if t.endswith('-1')]
    for t in multiples:
        base_title = t[:-2]
        matched_titles = [x for x in comparison.keys() if x.startswith(base_title) and len(x) <= len(t)]
        avg_metrics = dict()
        for k in comparison[matched_titles[0]].keys():
            if type(comparison[matched_titles[0]][k]) in (str, type(None)):
                # string metrics like choice, name, title etc are the same
                avg_metrics[k] = comparison[matched_titles[0]][k]
            else:
                avg_metrics[k] = np.mean([comparison[t][k] for t in matched_titles])

        averaged_comparison[base_title + f"-avg-{len(matched_titles)}"] = avg_metrics
        print(f"averaged {len(matched_titles)} models for {base_title}")
        for m in matched_titles:
            del comparison[m]
    comparison.update(averaged_comparison)
    return comparison

In [ ]:
# Extract performance metrics for comparison
all_comparison_metrics = extract_comparison_metrics(RESULTS)
comparison_metrics = consolidate_reruns(all_comparison_metrics)

metrics_of_interest = list(comparison_metrics.values())[0].keys()
print(metrics_of_interest)
comparison_df = pd.DataFrame(index=metrics_of_interest)

for model_name, metrics in comparison_metrics.items():
    model_values = [metrics.get(metric, np.nan) for metric in metrics_of_interest]
    comparison_df[model_name] = model_values

# Display the comparison table
comparison_df

## Performance Metric Comparisons

In [1182]:
from typing import Callable, Optional


def plot_metric_comparison(comparison_df: pd.DataFrame, metric_name: str, title: Optional[str]=None, lower_is_better=True, groupby: Optional[str] = None, filterby: Optional[tuple[str, Callable]] = None):
    plt.figure(figsize=(10, 6))
    
    # Get the data for this metric
    if filterby is not None:
        comparison_df = comparison_df.T[comparison_df.T.apply(filterby, axis=1)].T

    if groupby is not None:
        comparison_df = comparison_df.T.groupby([groupby])[[metric_name]].agg('mean').T

    metric_values = comparison_df.loc[metric_name]
    
    # Sort models by metric value
    if lower_is_better:
        sorted_models = metric_values.sort_values().index
    else:
        sorted_models = metric_values.sort_values(ascending=False).index
    
    # Create bar plot
    bars = plt.bar(sorted_models, metric_values[sorted_models])
    
    # Add value labels on top of bars with smaller font size
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.001,
                f'{height:.4f}', ha='center', va='bottom', rotation=0, fontsize=8)
    
    # Add title and labels
    plt.title(title or f'{metric_name.upper()} Comparison Across Models')
    plt.ylabel(metric_name)
    if groupby is not None:
        plt.xlabel(g)
    
    # Set rotated tick labels with proper alignment
    plt.xticks(rotation=45, ha='right')
    
    plt.tight_layout()
    
    return plt

In [1179]:
GROUPBY_OPTIONS = ['model_name', 'history_length', 'history_choice', 'expected_frames']

In [ ]:
# Plot MSE comparison (lower is better)
filterby = lambda x: "claude" in x['model_name'] and 'first' in x['history_choice']

for g in GROUPBY_OPTIONS:
    plot_metric_comparison(comparison_df, 'mse', 'Mean Squared Error (lower is better)' + f", grouped by {g}", groupby=g, filterby=filterby)

In [ ]:
comparison_df.T.columns

In [ ]:
from __future__ import annotations
import math
import pandas as pd
import matplotlib.pyplot as plt
import typing as t

def plot_mse_by_model(df: pd.DataFrame) -> None:
    """
    Single fig, subplots in a square(ish) grid.
    x = history_length
    y = mse
    color = history_choice
    legend lives outside on the right with handles from all subplots.
    """
    if "model_name" not in df.columns:
        raise ValueError("df needs column 'model_name'")

    models = sorted(df["model_name"].unique())
    n_mod = len(models)
    side = math.ceil(math.sqrt(n_mod))          # min grid side for square
    choices = sorted(df["history_choice"].unique())
    palette = plt.rcParams["axes.prop_cycle"].by_key()["color"]
    cdict = {c: palette[i % len(palette)] for i, c in enumerate(choices)}

    fig, axes = plt.subplots(
        nrows=side,
        ncols=side,
        figsize=(6 * side, 6 * side),
        sharex=True,
        sharey=False,
    )

    # flatten for easy indexing; hide unused pads
    axes_flat = axes.ravel()
    for ax in axes_flat[n_mod:]:
        ax.set_visible(False)

    # Collect all handles and labels from all subplots
    all_handles: t.Dict[str, t.Any] = {}
    all_labels: t.Set[str] = set()

    for idx, (ax, model) in enumerate(zip(axes_flat, models)):
        sdf = df[df["model_name"] == model]
        for choice, sub in sdf.groupby("history_choice"):
            scatter = ax.scatter(
                sub["history_length"],
                sub["mse"],
                s=sub["expected_frames"].values.astype(float) * 100,
                alpha=0.8,
                color=cdict[choice],
                label=choice,
            )
            # Store the handle for this choice if we haven't seen it yet
            if choice not in all_handles:
                all_handles[choice] = scatter
                all_labels.add(choice)
        
        ax.set_title(model)
        if idx % side == 0:
            ax.set_ylabel("mse")
        if idx // side == side - 1:
            ax.set_xlabel("history_len (# frames)")

    # Create the legend using all unique handles
    unique_handles = [all_handles[label] for label in sorted(all_labels)]
    unique_labels = sorted(all_labels)
    
    fig.legend(
        unique_handles,
        unique_labels,
        title="history_choice",
        loc="center left",
        bbox_to_anchor=(1.02, 0.5),
        borderaxespad=0.0,
        fontsize="x-small",
    )

    plt.tight_layout()
    plt.subplots_adjust(right=0.8)   # leave space for legend
    plt.show()

plot_mse_by_model(comparison_df.T)

In [ ]:
from __future__ import annotations
import pandas as pd
import matplotlib.pyplot as plt

def plot_history_expected_bubble(
    df: pd.DataFrame,
    size_scale: float = 1000.0,
) -> None:
    """
    single‑panel scatter:
        x  = history_length
        y  = expected_frames
        size = mse * size_scale  (bigger bubble == bigger error)
        color = model_name
    """
    required = {"model_name", "history_length", "expected_frames", "mse"}
    if not required.issubset(df.columns):
        missing = required - set(df.columns)
        raise ValueError(f"missing columns: {missing}")

    # fixed color palette keyed by model_name
    palette = plt.rcParams["axes.prop_cycle"].by_key()["color"]
    models   = sorted(df["model_name"].unique())
    cmap     = {m: palette[i % len(palette)] for i, m in enumerate(models)}

    fig, ax = plt.subplots(figsize=(6, 4))
    for model, sub in df.groupby("model_name"):
        print(sub['expected_frames'].values)
        print(sub['history_length'].values)
        print(sub['mse'].values)
        
        ax.scatter(
            sub["history_length"],
            sub["expected_frames"],
            s=sub["mse"].values.astype(float) * size_scale,
            alpha=0.7,
            color=cmap[model],
            label=model,
            edgecolors="none",
        )

    ax.set_xlabel("history_len (# frames)")
    ax.set_ylabel("expected_frames")
    ax.set_title("history_len vs expected_frames (bubble size = mse)")
    ax.legend(title="model_name", bbox_to_anchor=(1.02, 0.5), loc="center left", fontsize="x-small")
    plt.tight_layout()
    plt.subplots_adjust(right=0.8)
    plt.show()

plot_history_expected_bubble(comparison_df.T)

In [ ]:
from __future__ import annotations

"""plot_mse_by_model_single: one‑axes scatter with tri‑partite legend.

x              → history_length
y              → mse
marker (shape) → model_name
color          → history_choice
size           → expected_frames (scaled)

legend columns (all outside right):
    history_choice  (color swatches)
    expected_frames (size examples)
    model_name      (marker shapes)
"""

import math
from typing import Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D


MARKERS = [
    "o", "s", "^", "D", "v",
    "P", "X", "*", "<", ">",
    "h", "H", "p", "8",
]


def _build_marker_map(models: list[str]) -> Dict[str, str]:
    if len(models) > len(MARKERS):
        raise ValueError("too many models for built‑in marker set; extend MARKERS list")
    return {m: MARKERS[i] for i, m in enumerate(models)}


def plot_mse_by_model_single(df: pd.DataFrame, *, size_scale: float = 20.0) -> None:
    """draw scatter plot where mse is plotted against history_length across models."""

    required = {"model_name", "history_length", "mse", "history_choice", "expected_frames"}
    if not required.issubset(df.columns):
        raise ValueError(f"df missing columns: {required.difference(df.columns)}")

    # encode aesthetics
    models = sorted(df["model_name"].unique())
    marker_map = _build_marker_map(models)

    choices = sorted(df["history_choice"].unique())
    colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
    cdict: Dict[str, str] = {c: colors[i % len(colors)] for i, c in enumerate(choices)}

    # plot
    fig, ax = plt.subplots(figsize=(12,12))
    for _, row in df.iterrows():
        ax.scatter(
            int(row["history_length"]),
            row["mse"],
            s=row["expected_frames"] * size_scale,
            marker=marker_map[row["model_name"]],
            color=cdict[row["history_choice"]],
            alpha=0.8,
        )

    ax.set_xlabel("history_len (# frames)")
    ax.set_ylabel("mse")
    ax.set_title("mse vs history_len across models")

    # build legend handles -------------------------------------------------
    # 1) color legend (history_choice)
    color_handles = [
        Line2D([], [], marker="o", linestyle="", markerfacecolor=cdict[c], markeredgecolor="none", markersize=10, label=c)
        for c in choices
    ]

    # 2) size legend (expected_frames quartiles)
    q25, q50, q75 = np.quantile(df["expected_frames"], [0.25, 0.5, 0.75])
    size_handles = [
        ax.scatter([], [], s=q * size_scale, color="gray", alpha=0.6, label=f"{int(q)} frames")
        for q in (q25, q50, q75)
    ]

    # 3) marker legend (model_name)
    marker_handles = [
        Line2D([], [], marker=marker_map[m], linestyle="", color="gray", markersize=10, label=m)
        for m in models
    ]

    # place legends outside right
    legend_kw = dict(loc="upper left", bbox_to_anchor=(1.02, 1), frameon=False)
    first = ax.legend(handles=color_handles, title="history_choice", **legend_kw)
    ax.add_artist(first)

    legend_kw["bbox_to_anchor"] = (1.02, 0.5)
    second = ax.legend(handles=size_handles, title="expected_frames", **legend_kw)
    ax.add_artist(second)

    legend_kw["bbox_to_anchor"] = (1.02, 0)
    ax.legend(handles=marker_handles, title="model_name", **legend_kw)

    plt.tight_layout()
    plt.show()

plot_mse_by_model_single(comparison_df.T)

In [ ]:
from __future__ import annotations

"""plot_mse_by_model
====================
flexible visualisation of mse vs history_length with robust sizing logic.

changes vs previous rev
-----------------------
* ``expected_frames`` may contain **zero** → size fallback of *1 frame* so the dot is visible.
* size legend automatically deduplicates values and spans the whole numeric range, incl. ``0`` when present.
* public API unchanged: ``plot_mse_by_model(df, grid=False, size_scale=20.0)``.

layout recap
------------
* one‑axes (default) or N×N grid (``grid=True``).
* marker → model_name, color → history_choice, size → expected_frames.
* tri‑column legend outside right for color, size, marker.
"""

from typing import Dict, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# ----------------------------- aesthetics helpers -----------------------------
MARKERS: List[str] = [
    "o", "s", "^", "D", "v",
    "P", "X", "*", "<", ">",
    "h", "H", "p", "8",
]


def _build_marker_map(models: List[str]) -> Dict[str, str]:
    if len(models) > len(MARKERS):
        raise ValueError("too many models for built‑in marker set; extend MARKERS list")
    return {m: MARKERS[i] for i, m in enumerate(models)}


# ----------------------------- size helpers -----------------------------

def _effective_size(frames: float, scale: float) -> float:
    """convert expected_frames → marker size in points²; zero gets minimal dot."""
    return max(frames, 1.0) * scale  # 0 → 1


def _legend_size_values(frames: np.ndarray) -> List[int]:
    """choose up‑to‑4 representative frame counts for legend.

    * always include 0 when present.
    * include min, median, max of non‑zero values (deduped).
    """
    frames = frames[~np.isnan(frames)]
    zeros_present = np.any(frames == 0)
    nonzero = frames[frames > 0]
    legend_vals: List[int] = []

    if zeros_present:
        legend_vals.append(0)

    if nonzero.size:
        # pick min, median, max unique ints
        q_vals = [np.min(nonzero), np.median(nonzero), np.max(nonzero)]
        for v in q_vals:
            iv = int(round(v))
            if iv not in legend_vals:
                legend_vals.append(iv)

    return legend_vals[:4]  # cap to 4 entries


# ----------------------------- single‑axes plot -----------------------------

def _plot_single(df: pd.DataFrame, marker_map: Dict[str, str], cdict: Dict[str, str], *, size_scale: float) -> None:
    fig, ax = plt.subplots(figsize=(10, 8))

    for _, row in df.iterrows():
        ax.scatter(
            row["history_length"],
            row["mse"],
            s=_effective_size(row["expected_frames"], size_scale),
            marker=marker_map[row["model_name"]],
            color=cdict[row["history_choice"]],
            alpha=0.8,
        )

    ax.set_xlabel("history_len (# frames)")
    ax.set_ylabel("mse")
    ax.set_title("mse vs history_len across models")

    _add_legends(ax, df, marker_map, cdict, size_scale)
    plt.tight_layout()
    plt.show()


# ----------------------------- grid plot -----------------------------

def _plot_grid(df: pd.DataFrame, marker_map: Dict[str, str], cdict: Dict[str, str], *, size_scale: float) -> None:
    import math

    models = sorted(df["model_name"].unique())
    n_mod = len(models)
    side = math.ceil(math.sqrt(n_mod))

    fig, axes = plt.subplots(
        nrows=side,
        ncols=side,
        figsize=(6 * side, 6 * side),
        sharex=True,
        sharey=False,
    )

    axes_flat = axes.ravel()
    for ax in axes_flat[n_mod:]:
        ax.set_visible(False)

    for idx, (ax, model) in enumerate(zip(axes_flat, models)):
        sdf = df[df["model_name"] == model]
        for choice, sub in sdf.groupby("history_choice"):
            ax.scatter(
                sub["history_length"],
                sub["mse"],
                s=_effective_size(sub["expected_frames"], size_scale),
                marker=marker_map[model],
                color=cdict[choice],
                alpha=0.8,
            )
        ax.set_title(model)
        if idx % side == 0:
            ax.set_ylabel("mse")
        if idx // side == side - 1:
            ax.set_xlabel("history_len (# frames)")

    _add_legends(axes_flat[0], df, marker_map, cdict, size_scale)
    plt.tight_layout()
    plt.subplots_adjust(right=0.8)
    plt.show()


# ----------------------------- legend builder -----------------------------

def _add_legends(ax, df: pd.DataFrame, marker_map: Dict[str, str], cdict: Dict[str, str], size_scale: float) -> None:
    choices = sorted(df["history_choice"].unique())

    # 1) color legend (history_choice)
    color_handles = [
        Line2D([], [], marker="o", linestyle="", markerfacecolor=cdict[c], markeredgecolor="none", markersize=10, label=c)
        for c in choices
    ]

    # 2) size legend (expected_frames)
    expected_frames = df["expected_frames"]
    size_vals = _legend_size_values(expected_frames.astype(float).values)
    size_handles = [
        ax.scatter([], [], s=_effective_size(v, size_scale), color="gray", alpha=0.6, label=f"{v} frames")
        for v in size_vals
    ]

    # 3) marker legend (model_name)
    marker_handles = [
        Line2D([], [], marker=marker_map[m], linestyle="", color="gray", markersize=10, label=m)
        for m in sorted(marker_map)
    ]

    legend_kw = dict(loc="upper left", bbox_to_anchor=(1.02, 1), frameon=False)
    first = ax.legend(handles=color_handles, title="history_choice", **legend_kw)
    ax.add_artist(first)

    legend_kw["bbox_to_anchor"] = (1.02, 0.5)
    second = ax.legend(handles=size_handles, title="expected_frames", **legend_kw)
    ax.add_artist(second)

    legend_kw["bbox_to_anchor"] = (1.02, 0)
    ax.legend(handles=marker_handles, title="model_name", **legend_kw)


# ----------------------------- public API -----------------------------

def plot_mse_by_model(
    df: pd.DataFrame,
    *,
    grid: bool = False,
    size_scale: float = 20.0,
) -> None:
    """plot mse vs history_length.

    parameters
    ----------
    df : pd.DataFrame
        requires columns ``{"model_name", "history_length", "mse", "history_choice", "expected_frames"}``.
    grid : bool, default ``False``
        True → one subplot per model in a square grid.
    size_scale : float, default ``20.0``
        multiplier converting frames → marker points²; zero frames mapped to size of *1 frame*.
    """

    required = {"model_name", "history_length", "mse", "history_choice", "expected_frames"}
    if not required.issubset(df.columns):
        raise ValueError(f"df missing columns: {required.difference(df.columns)}")

    models = sorted(df["model_name"].unique())
    marker_map = _build_marker_map(models)

    choices = sorted(df["history_choice"].unique())
    colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
    cdict: Dict[str, str] = {c: colors[i % len(colors)] for i, c in enumerate(choices)}

    if grid:
        _plot_grid(df, marker_map, cdict, size_scale=size_scale)
    else:
        _plot_single(df, marker_map, cdict, size_scale=size_scale)


plot_mse_by_model(comparison_df.T, grid=False)

In [ ]:
comparison_df.T[comparison_df.T['history_length'] == 0]

In [ ]:
comparison_df.T[comparison_df.T['mse'] < .02]

In [1124]:
# Plot Cosine Similarity comparison (higher is better)
# plot_metric_comparison(comparison_df, 'cosine_sim', 'Cosine Similarity (higher is better)', lower_is_better=False)

In [1125]:
# Plot Inference Time comparison (lower is better)
if 'mean_inference_time' in comparison_df.index:
    # Convert to milliseconds for better readability
    inference_times_ms = comparison_df.loc['mean_inference_time'] * 1000
    
    plt.figure(figsize=(10, 6))
    
    # Sort models by inference time
    sorted_models = inference_times_ms.sort_values().index
    
    # Create bar plot
    bars = plt.bar(sorted_models, inference_times_ms[sorted_models])
    
    # Add value labels on top of bars with smaller font size
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{height:.1f} ms', ha='center', va='bottom', rotation=0, fontsize=8)
    
    plt.title('Mean Inference Time (lower is better)')
    plt.ylabel('Time (ms)')
    plt.xlabel('Model')
    
    # Set rotated tick labels with proper alignment
    plt.xticks(rotation=45, ha='right')
    
    plt.tight_layout()

## Action Magnitude Analysis

In [ ]:
# Improved action magnitude comparison plot with simple color scheme
# Extract data
models = list(comparison_metrics.keys())
pred_mags = [comparison_metrics[model]['pred_mag'] for model in models]
gt_mag = comparison_metrics[models[0]]['gt_mag']  # Ground truth should be the same for all models

# Set up a simple color palette
colors = plt.cm.tab10(np.arange(len(models)))

# Create plot
fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.bar(np.arange(len(models)), pred_mags, color=colors, alpha=0.8)

# Add horizontal line for ground truth
ax.axhline(y=gt_mag, color='red', linestyle='--', alpha=0.7, label=f'Ground Truth: {gt_mag:.4f}')

# Add value labels on top of bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., 
            height + 0.01,
            f'{height:.4f}', 
            ha='center', 
            va='bottom',
            fontsize=10)

# Set labels and title
ax.set_xlabel('Model')
ax.set_ylabel('Action Magnitude')
ax.set_title('Action Magnitude Comparison')
ax.set_xticks(np.arange(len(models)))
ax.set_xticklabels(models, rotation=45, ha='right')
ax.legend()

# Add grid for better readability
ax.grid(axis='y', linestyle='--', alpha=0.3)

plt.tight_layout()

## Trajectory-level Analysis

In [1127]:
# Function to extract trajectory-level metrics for all models
def get_trajectory_metrics(model_data, metric_name='mse'):
    traj_metrics = {}
    
    for model_name, data in model_data.items():
        # If we have pre-computed metrics, use those
        if 'metrics' in data and 'results' in data['metrics']:
            traj_metrics[model_name] = [r.get(metric_name, np.nan) for r in data['metrics']['results']]
        else:
            # Otherwise, compute metrics from actions
            print(f"Computing trajectory metrics for {model_name}...")
            model_traj_metrics = []
            
            for traj in data['actions']:
                pred_actions = traj['pred_actions']
                gt_actions = traj['gt_actions']
                
                if metric_name == 'mse':
                    value = np.mean(np.square(pred_actions - gt_actions))
                elif metric_name == 'mae':
                    value = np.mean(np.abs(pred_actions - gt_actions))
                else:
                    # Compute the requested metric
                    results = compute_metrics(pred_actions, gt_actions)
                    value = results.get(metric_name, np.nan)
                    
                model_traj_metrics.append(value)
            
            traj_metrics[model_name] = model_traj_metrics
    
    return traj_metrics

In [ ]:
# Get MSE for each trajectory across models
traj_mse = get_trajectory_metrics(model_data, 'mse')

# Calculate average and median MSE for each trajectory (across all models)
traj_indices = range(len(next(iter(traj_mse.values()))))
avg_per_traj = [np.mean([traj_mse[model][i] for model in traj_mse.keys()]) for i in traj_indices]
median_per_traj = [np.median([traj_mse[model][i] for model in traj_mse.keys()]) for i in traj_indices]

# Plot trajectory-level MSE comparison
plt.figure(figsize=(14, 8))

# Plot individual model lines
for model_name, mse_values in traj_mse.items():
    plt.plot(traj_indices, mse_values, marker='o', alpha=0.7, linewidth=1, label=model_name)

# Plot average and median with thicker lines for emphasis
plt.plot(traj_indices, avg_per_traj, 'k-', linewidth=3, label='Average MSE')
plt.plot(traj_indices, median_per_traj, 'r--', linewidth=3, label='Median MSE')

plt.title('MSE per Trajectory Across Models')
plt.xlabel('Trajectory Index')
plt.ylabel('MSE')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(True)
plt.tight_layout()

In [ ]:
# Create boxplot for comparison of MSE distribution across models
plt.figure(figsize=(12, 6))

# Prepare data for boxplot
box_data = [values for model, values in traj_mse.items()]
box_labels = list(traj_mse.keys())

plt.boxplot(box_data, labels=box_labels, patch_artist=True)
plt.title('Distribution of MSE Across Trajectories')
plt.ylabel('MSE')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

## Dimension-wise Action Analysis

In [1130]:
# Analyze MSE by action dimension
def compute_dimension_mse(model_data):
    dim_mse = {}
    
    # First, get action dimensions from the first model
    first_model = next(iter(model_data.values()))
    first_traj = first_model['actions'][0]
    action_dims = first_traj['pred_actions'].shape[1]
    
    for model_name, data in model_data.items():
        dim_mse[model_name] = []
        
        for dim in range(action_dims):
            dim_errors = []
            
            for traj in data['actions']:
                pred_dim = traj['pred_actions'][:, dim]
                gt_dim = traj['gt_actions'][:, dim]
                dim_errors.append(np.mean(np.square(pred_dim - gt_dim)))
            
            dim_mse[model_name].append(np.mean(dim_errors))
    
    return dim_mse, action_dims

In [ ]:
# Plot dimension-wise MSE on separate subplots with improved styling
def plot_dimension_mse_subplots(dim_mse, action_dims):
    # Determine number of rows and columns for subplots
    ncols = 2
    nrows = (action_dims + ncols - 1) // ncols  # Ceiling division
    
    # Create figure with subplots
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 4*nrows))
    axes = axes.flatten() if nrows > 1 or ncols > 1 else [axes]
    
    # Define a colormap for consistent colors across models
    colors = plt.cm.tab10.colors  # Get colors from tab10 colormap
    model_names = list(dim_mse.keys())
    model_colors = {model: colors[i % len(colors)] for i, model in enumerate(model_names)}
    
    # Plot each dimension in its own subplot
    for dim in range(action_dims):
        ax = axes[dim]
        
        # Extract MSE values for this dimension across all models
        mse_values = [dim_mse[model][dim] for model in model_names]
        
        # Create bar plot for this dimension with custom colors
        bars = ax.bar(model_names, mse_values, color=[model_colors[model] for model in model_names])
        
        # Add value labels on top of bars with better formatting
        for bar in bars:
            height = bar.get_height()
            # Format the text based on the magnitude
            if height < 0.001:
                text = f'{height:.2e}'  # Scientific notation for very small values
            elif height < 0.01:
                text = f'{height:.5f}'
            else:
                text = f'{height:.3f}'
                
            ax.text(bar.get_x() + bar.get_width()/2., height * 1.02,
                   text, ha='center', va='bottom', fontsize=10, 
                   rotation=0, fontweight='bold', color='black')
        
        # Set title and labels with better styling
        ax.set_title(f'Dimension {dim}', fontsize=14, pad=10)
        ax.set_ylabel('MSE', fontsize=12)
        
        # Improve x-tick labels
        ax.set_xticklabels(model_names, rotation=30, ha='right', fontsize=10)
        
        # Improve grid and background
        ax.grid(axis='y', linestyle='--', alpha=0.3)
        ax.set_axisbelow(True)  # Put grid lines behind bars
        
        # Add light background color
        ax.set_facecolor('#f8f8f8')
    
    # Hide any unused subplots
    for i in range(action_dims, len(axes)):
        axes[i].set_visible(False)
    
    # Add a single legend for all subplots
    handles = [plt.Rectangle((0,0), 1, 1, color=model_colors[model]) for model in model_names]
    fig.legend(handles, model_names, loc='upper center', bbox_to_anchor=(0.5, 0), 
              ncol=min(5, len(model_names)), fontsize=12, frameon=True, facecolor='white')
    
    plt.tight_layout()
    plt.suptitle('MSE by Action Dimension (Each plot has its own scale)', 
                y=1.02, fontsize=16, fontweight='bold')
    plt.subplots_adjust(top=0.92, bottom=0.15)  # Make room for the legend at the bottom
    
    return fig

# Compute dimension-wise MSE
dim_mse, action_dims = compute_dimension_mse(model_data)
# Use this function to plot the dimension-wise MSE
plot_dimension_mse_subplots(dim_mse, action_dims)

In [ ]:
comparison_df

In [ ]:

pred_actions = {k: [tj['pred_actions'] for tj in v['actions']] for k,v in model_data.items()}
gt_actions = {k: [tj['gt_actions'] for tj in v['actions']] for k,v in model_data.items()}

pred_actions = {k: np.concatenate(v, axis=0 ) for k,v in pred_actions.items()}
gt_actions = {k: np.concatenate(v, axis=0) for k,v in gt_actions.items()}

for k in pred_actions.keys():
    print(f"pred: {k}, {pred_actions[k].shape}")
    print(f"gt: {k}, {gt_actions[k].shape}")


In [ ]:
def plot_action_dimensions(pred_actions, gt_actions, model_names=None):
    """
    Plot each dimension of the action data over time steps.
    
    Args:
        pred_actions: Dictionary mapping model names to predicted action arrays
        gt_actions: Dictionary mapping model names to ground truth action arrays
        model_names: Optional list of model names to plot (if None, plots all)
    """
    if model_names is None:
        model_names = list(pred_actions.keys())
    
    # Get the action dimensions from the first model
    first_model = next(iter(pred_actions.values()))
    action_dims = first_model.shape[1]
    
    # Create a figure for each action dimension
    for dim in range(action_dims):
        plt.figure(figsize=(12, 6))
        
        # Plot predicted actions for each model
        for model in model_names:
            # Get time steps for x-axis
            time_steps = np.arange(len(pred_actions[model]))
            
            # Plot predicted actions
            plt.plot(time_steps, pred_actions[model][:, dim], 
                     label=f"{model} (pred)", alpha=0.7)
            
            # Plot ground truth actions (only once since it's the same for all models)
            if model == model_names[0]:
                plt.plot(time_steps, gt_actions[model][:, dim], 
                         label="Ground Truth", color='black', linestyle='--', linewidth=2)
        
        plt.title(f"Action Dimension {dim}")
        plt.xlabel("Time Step")
        plt.ylabel("Action Value")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

plot_action_dimensions(pred_actions, gt_actions)

In [1135]:
def plot_metrics_vs_latency(comparison_df, metrics=None, figsize=(14, 10), save_path=None):
    """
    Create scatterplots of metrics vs latency
    
    Args:
        comparison_df: DataFrame with models as columns and metrics as rows
        metrics: List of metrics to plot against latency (default: ['mse'])
        figsize: Size of the figure (default: (14, 10))
        save_path: Path to save the plot (default: None)
    """
    if metrics is None:
        metrics = ['mse']
    
    # Transpose to get models as rows
    df_transposed = comparison_df.T
    
    # Convert latency to milliseconds
    df_transposed['latency_ms'] = df_transposed['mean_inference_time'] * 1000
    
    # Set up the plot grid
    n_metrics = len(metrics)
    n_cols = min(2, n_metrics)
    n_rows = (n_metrics + n_cols - 1) // n_cols  # Ceiling division
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize, squeeze=False)
    sns.set(style="whitegrid")
    
    # Flatten the axes array for easier indexing
    axes = axes.flatten()
    
    # Get unique colors for each model
    colors = sns.color_palette("husl", len(df_transposed))
    
    # Plot each metric
    for i, metric in enumerate(metrics):
        if i < len(axes):
            ax = axes[i]
            
            # Check if the metric exists in the DataFrame
            if metric not in df_transposed.columns:
                ax.text(0.5, 0.5, f"Metric '{metric}' not found", 
                        ha='center', va='center', transform=ax.transAxes)
                continue
            
            # Plot the points
            for j, (model, row) in enumerate(df_transposed.iterrows()):
                ax.scatter(
                    row['latency_ms'], 
                    row[metric], 
                    s=100,  # marker size
                    color=colors[j],
                    label=model,
                    alpha=0.7
                )
                
                # Add model name as text label
                ax.text(
                    row['latency_ms'] * 1.03,  # slightly offset
                    row[metric] * 1.03,  # slightly offset
                    model,
                    fontsize=9
                )
            
            # Add title and labels
            ax.set_title(f'{metric.upper()} vs Latency', fontsize=14)
            ax.set_xlabel('Latency (ms)', fontsize=12)
            ax.set_ylabel(metric.upper(), fontsize=12)
            
            # Rotate x-axis tick labels to prevent overlap
            plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
            
            # Add grid
            ax.grid(True, linestyle='--', alpha=0.7)
            
            # Annotate the ideal region (low metric, low latency)
            min_latency = df_transposed['latency_ms'].min()
            min_metric = df_transposed[metric].min()
            
            ax.annotate('Better', 
                       xy=(min_latency, min_metric), 
                       xytext=(min_latency + 5, min_metric * 0.7),
                       arrowprops=dict(facecolor='green', shrink=0.05, width=2),
                       fontsize=12, color='green')
    
    # Hide unused subplots
    for j in range(i+1, len(axes)):
        axes[j].axis('off')
    
    # Adjust layout with extra bottom margin for rotated labels
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    
    # Save if requested
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Plot saved as '{save_path}'")
    
    # Show the plot
    plt.show()

In [ ]:
plot_metrics_vs_latency(comparison_df, metrics=['mse', 'mae', 'r2', 'cosine_sim'])
